# **Задание 4**

# Библиотеки

In [ ]:
%%writefile requirements.txt
pytorch-lightning
scikit-learn
razdel
fasttext

Writing requirements.txt


In [ ]:
!pip install --upgrade -r requirements.txt

Requirement already up-to-date: pytorch-lightning in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 1)) (1.1.1)
Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 2)) (0.23.2)
Requirement already up-to-date: razdel in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 3)) (0.5.0)
Requirement already up-to-date: fasttext in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 4)) (0.9.2)


In [ ]:
!rm requirements.txt

# Датасет

In [ ]:
# все что про датасет взято из семинара без изменений
!rm negative.csv
!rm positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv

rm: cannot remove 'negative.csv': No such file or directory
rm: cannot remove 'positive.csv': No such file or directory
--2020-12-16 22:09:37--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/r6u59ljhhjdg6j0/negative.csv [following]
--2020-12-16 22:09:37--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb68e00d0eb0500f4a5d21f3eb1.dl.dropboxusercontent.com/cd/0/inline/BFNvUgnMQNYmtvpcYhuQYAZQ4r0ka86gsjaD5ajpIzcZNMFQ9A9RLLzfdWp-3COdBPfNOUdKQ-k5zVdwk5qXTvqh_VRDCHDZwlxBC1-5NNFiaA/file# [following]
--2020-12-16 22:09:38--  https://ucb68e00d0eb0500f4a5d21f3eb1.dl.dropboxusercontent.com/cd/0/inline/BFNvUgnMQNYm

In [ ]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['text'].values[:sample_size], data_negative['text'].values[:sample_size]), axis=0)

def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()

df_train = pd.DataFrame(columns=['text', 'label'])
df_val = pd.DataFrame(columns=['text', 'label'])
df_test = pd.DataFrame(columns=['text', 'label'])

data = [preprocess_text(t) for t in raw_data]
labels = [1] * sample_size + [0] * sample_size
df_train['text'], df_test['text'], df_train['label'], df_test['label'] = train_test_split(data, labels, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=1)
df_train

,text,label
85913,раньше все встречались у фонтана в гуме а тепе...,1
42792,ни когда не пойму любовь женщины к женщине хот...,1
85556,два сеанса в кино вот что я люблю парам пам па...,1
36360,я нашел мой наряд для кэти перри USER URL via ...,1
154940,USER ууууу всеее развод обидки и все дела,1
...,...,...
73349,закидываю свой вконтакторостер строчками из пе...,1
109259,гребаный понедельник гребанные 4 пары хочу дом...,0
50057,болячки вроде подживают но губы увеличелись в ...,0
5192,USER я знала что она добрая а за что она тебе,1


In [ ]:
del data_positive
del data_negative

In [ ]:
#!rm negative.csv
#!rm positive.csv

In [ ]:
from collections import Counter


class Vocabulary:
    def __init__(self):
        self.word2index = {
            "<pad>": 0,
            "<unk>": 1
        }
        self.index2word = ["<pad>", "<unk>"]

    def build(self, texts, min_count=7):
        words_counter = Counter(token for tokens in texts for token in tokens)
        for word, count in words_counter.most_common():
            if count >= min_count:
                self.word2index[word] = len(self.word2index)
        self.index2word = [word for word, _ in sorted(self.word2index.items(), key=lambda x: x[1])]
    
    def get_pad(self):
        return self.word2index["<pad>"]

    def get_unk(self):
        return self.word2index["<unk>"]
    
    @property
    def size(self):
        return len(self.index2word)
    
    def top(self, n=100):
        return self.index2word[1:n+1]
    
    def get_index(self, word):
        return self.word2index.get(word, self.get_unk())
    
    def get_word(self, index):
        return self.index2word[index]

# Задание 4.1
Сделайте char-level CNN для решения этой же задачи

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler


class TextDataset(Dataset):
    def __init__(self, tokenized_texts, labels, vocabulary, max_seq_len=200):
        self.samples = []
        assert len(tokenized_texts) != 0
        assert len(tokenized_texts) == len(labels)
        for text, label in zip(tokenized_texts, labels):
            indices = [vocabulary.get_index(word) for word in text][:max_seq_len]
            indices += [vocabulary.get_pad() for _ in range(max_seq_len - len(indices))]
            self.samples.append((torch.LongTensor(indices), torch.FloatTensor([label])))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        return self.samples[index]


In [ ]:
import os
import torch
import random
import numpy as np

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(42)

In [ ]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule
from pytorch_lightning.metrics import Accuracy
 

class SimpleModel(LightningModule):
    def __init__(self, vocab_size, embedding_dim=32):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(vocab_size, embedding_dim)
        self.loss = nn.BCEWithLogitsLoss()
        self.valid_accuracy = Accuracy()
        self.test_accuracy = Accuracy()
    
    def forward(self, inputs, labels):
        raise NotImplementedError("forward not implemented")
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return [optimizer]
    
    def training_step(self, batch, _):
        inputs, labels = batch
        loss, logits = self(inputs, labels)
        return loss
    
    def validation_step(self, batch, _):
        inputs, labels = batch
        val_loss, logits = self(inputs, labels)
        self.valid_accuracy.update(logits, labels)
        self.log("val_loss", val_loss, prog_bar=True)
        self.log("val_acc", self.valid_accuracy)

    def validation_epoch_end(self, outs):
        self.log("val_acc_epoch", self.valid_accuracy.compute(), prog_bar=True)

    def test_step(self, batch, _):
        inputs, labels = batch
        test_loss, logits = self(inputs, labels)
        self.test_accuracy.update(logits, labels)
        self.log("test_loss", test_loss, prog_bar=True)
        self.log("test_acc", self.test_accuracy)

    def test_epoch_end(self, outs):
        self.log("test_acc_epoch", self.test_accuracy.compute(), prog_bar=True)

In [ ]:
import torch
import torch.nn as nn
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping  

class CnnModel(SimpleModel):
    def __init__(self, vocab_size, embedding_dim=128, filters=((2, 10), (3, 8))):
        super().__init__(vocab_size, embedding_dim)
        
        self.filters = []
        all_filters_count = 0
        for kernel_size, filters_count in filters:
            all_filters_count += filters_count
            self.filters.append(nn.Conv2d(1, filters_count, (kernel_size, embedding_dim), padding=(1, 0)))
        self.filters = nn.ModuleList(self.filters)
        self.relu_layer = nn.ReLU()
        self.dropout_layer = nn.Dropout(0.2)
        self.out_layer = nn.Linear(all_filters_count, 1)
    
    def forward(self, inputs, labels):
        projections = self.embeddings_layer.forward(inputs)
        # batch_size x num_words x embedding_dim
        projections = projections.unsqueeze(1)
        # batch_size x 1 x num_words x embedding_dim
        results = []
        for f in self.filters:
            convolved = self.dropout_layer(self.relu_layer(f(projections))).squeeze(3)
            # batch_size x num_filters x num_words
            pooling = torch.max(convolved, 2)[0]
            # batch_size x num_filters
            results.append(pooling)
        output = torch.cat(results, 1)
        # batch_sisze x sum(num_filters)
        # 128 x 18
        logits = self.out_layer.forward(output)
        loss = self.loss(logits, labels)
        return loss, logits

In [ ]:
train_labels = df_train["label"].tolist()
val_labels = df_val["label"].tolist()
test_labels = df_test["label"].tolist()

texts_set = set(df_train["text"].tolist()) | set(df_val["text"].tolist()) | set(df_test["text"].tolist())

# [token.text for token in tokenize(text)] -> list(text)
# т.к. теперь разбиваем на символы, а не токены 
train_texts = [list(text) for text in df_train["text"].tolist()]
val_texts = [list(text) for text in df_val["text"].tolist()]
test_texts = [list(text) for text in df_test["text"].tolist()]

In [ ]:
# проверим, что текст разбился на символы
test_texts[0][:10]

['U', 'S', 'E', 'R', ' ', 'н', 'у', ' ', 'т', 'ы']

In [ ]:
vocabulary = Vocabulary()
vocabulary.build(train_texts)
print(vocabulary.size)
print(vocabulary.top(10))

75
['<unk>', ' ', 'о', 'а', 'е', 'т', 'н', 'и', 'с', 'л']


In [ ]:
BATCH_SIZE = 128

train_data = TextDataset(train_texts, train_labels, vocabulary)
train_sampler = RandomSampler(train_data)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, sampler=train_sampler)

val_data = TextDataset(val_texts, val_labels, vocabulary)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

test_data = TextDataset(test_texts, test_labels, vocabulary)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [ ]:
cnn_model = CnnModel(vocabulary.size, embedding_dim=50)
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=1,
    verbose=True,
    mode="min" 
)
trainer = Trainer(
    gpus=1,
    checkpoint_callback=False,
    accumulate_grad_batches=1,
    max_epochs=10,
    progress_bar_refresh_rate=10,
    callbacks=[early_stop_callback])
trainer.fit(cnn_model, train_loader, val_loader)
trainer.test(cnn_model, test_loader)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type              | Params
-------------------------------------------------------
0 | embeddings_layer | Embedding         | 3.8 K 
1 | loss             | BCEWithLogitsLoss | 0     
2 | valid_accuracy   | Accuracy          | 0     
3 | test_accuracy    | Accuracy          | 0     
4 | filters          | ModuleList        | 2.2 K 
5 | relu_layer       | ReLU              | 0     
6 | dropout_layer    | Dropout           | 0     
7 | out_layer        | Linear            | 19    
-------------------------------------------------------
6.0 K     Trainable params
0         Non-trainable para


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.6733, device='cuda:0'),
 'test_acc_epoch': tensor(0.6733, device='cuda:0'),
 'test_loss': tensor(0.5672, device='cuda:0'),
 'val_acc': tensor(0.6744, device='cuda:0'),
 'val_acc_epoch': tensor(0.6744, device='cuda:0'),
 'val_loss': tensor(0.5670, device='cuda:0')}
--------------------------------------------------------------------------------


[{'test_acc': 0.6733303666114807,
  'test_acc_epoch': 0.6733303666114807,
  'test_loss': 0.5671661496162415,
  'val_acc': 0.6744471788406372,
  'val_acc_epoch': 0.6744471788406372,
  'val_loss': 0.5669726133346558}]

In [ ]:
# возьм фильтры большего размера, т.к. у символов должен быть больший контекст
cnn_model2 = CnnModel(vocabulary.size, embedding_dim=50, filters=((10,10), (8,8), (6, 6)))
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=1,
    verbose=True,
    mode="min" 
)
trainer = Trainer(
    gpus=1,
    checkpoint_callback=False,
    accumulate_grad_batches=1,
    max_epochs=10,
    progress_bar_refresh_rate=10,
    callbacks=[early_stop_callback])
trainer.fit(cnn_model2, train_loader, val_loader)
trainer.test(cnn_model2, test_loader)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type              | Params
-------------------------------------------------------
0 | embeddings_layer | Embedding         | 3.8 K 
1 | loss             | BCEWithLogitsLoss | 0     
2 | valid_accuracy   | Accuracy          | 0     
3 | test_accuracy    | Accuracy          | 0     
4 | filters          | ModuleList        | 10.0 K
5 | relu_layer       | ReLU              | 0     
6 | dropout_layer    | Dropout           | 0     
7 | out_layer        | Linear            | 25    
-------------------------------------------------------
13.8 K    Trainable params
0         Non-trainable para


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.7312, device='cuda:0'),
 'test_acc_epoch': tensor(0.7312, device='cuda:0'),
 'test_loss': tensor(0.5198, device='cuda:0'),
 'val_acc': tensor(0.7345, device='cuda:0'),
 'val_acc_epoch': tensor(0.7345, device='cuda:0'),
 'val_loss': tensor(0.5184, device='cuda:0')}
--------------------------------------------------------------------------------


[{'test_acc': 0.7311592698097229,
  'test_acc_epoch': 0.7311592698097229,
  'test_loss': 0.5198193788528442,
  'val_acc': 0.7345041036605835,
  'val_acc_epoch': 0.7345041036605835,
  'val_loss': 0.5184457302093506}]

In [ ]:
cnn_model3 = CnnModel(vocabulary.size, embedding_dim=50, filters=((15,15), (10, 10), (8,8), (6, 6)))
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=1,
    verbose=True,
    mode="min" 
)
trainer = Trainer(
    gpus=1,
    checkpoint_callback=False,
    accumulate_grad_batches=1,
    max_epochs=10,
    progress_bar_refresh_rate=10,
    callbacks=[early_stop_callback])
trainer.fit(cnn_model3, train_loader, val_loader)
trainer.test(cnn_model3, test_loader)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type              | Params
-------------------------------------------------------
0 | embeddings_layer | Embedding         | 3.8 K 
1 | loss             | BCEWithLogitsLoss | 0     
2 | valid_accuracy   | Accuracy          | 0     
3 | test_accuracy    | Accuracy          | 0     
4 | filters          | ModuleList        | 21.3 K
5 | relu_layer       | ReLU              | 0     
6 | dropout_layer    | Dropout           | 0     
7 | out_layer        | Linear            | 40    
-------------------------------------------------------
25.1 K    Trainable params
0         Non-trainable para


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.7370, device='cuda:0'),
 'test_acc_epoch': tensor(0.7370, device='cuda:0'),
 'test_loss': tensor(0.5181, device='cuda:0'),
 'val_acc': tensor(0.7422, device='cuda:0'),
 'val_acc_epoch': tensor(0.7422, device='cuda:0'),
 'val_loss': tensor(0.5154, device='cuda:0')}
--------------------------------------------------------------------------------


[{'test_acc': 0.7370113730430603,
  'test_acc_epoch': 0.7370113730430603,
  'test_loss': 0.5181421041488647,
  'val_acc': 0.7421543598175049,
  'val_acc_epoch': 0.7421543598175049,
  'val_loss': 0.5154470205307007}]

Для сравнения такой результат был у модели с семинара после 6 эпох:

```
[{'test_acc': 0.7471074461936951,
  'test_acc_epoch': 0.7471074461936951,
  'test_loss': 0.5175567269325256,
  'val_acc': 0.7491623759269714,
  'val_acc_epoch': 0.7491623759269714,
  'val_loss': 0.5164740681648254}]
```



# Задание 4.2
Сделайте RNN с другими готовыми эмбеддингами (не обязательно fastText) для решения этой же задачи

## Общая часть с построением модели и датасета

In [ ]:
import razdel
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, RandomSampler

class FastTextDataset(Dataset):
    def __init__(self, tokenized_texts, labels, ft_model, max_seq_len=50):
        self.ft_model = ft_model
        self.vector_dim = self.ft_model.get_dimension()
        self.samples = []
        assert len(tokenized_texts) != 0
        assert len(tokenized_texts) == len(labels)
        for i, (text, label) in enumerate(zip(tokenized_texts, labels)):
            text = text[:max_seq_len]
            embeddings = np.zeros((max_seq_len, self.vector_dim))
            embeddings[:len(text), :] = self.words_to_embeddings(text)
            self.samples.append((torch.FloatTensor(embeddings), torch.FloatTensor([label])))
            if i % 10000 == 0:
                print(i)

    def words_to_embeddings(self, words):
        vector_dim = self.ft_model.get_dimension()
        embeddings = np.zeros((len(words), vector_dim))
        for i, w in enumerate(words):
            embeddings[i] = self.ft_model.get_word_vector(w)
            embeddings[i] /= np.linalg.norm(embeddings[i])
        return embeddings
        
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        return self.samples[index]

In [ ]:
from razdel import tokenize

train_labels = df_train["label"].tolist()
val_labels = df_val["label"].tolist()
test_labels = df_test["label"].tolist()

texts_set = set(df_train["text"].tolist()) | set(df_val["text"].tolist()) | set(df_test["text"].tolist())

train_texts = [[token.text for token in tokenize(text)] for text in df_train["text"].tolist()]
val_texts = [[token.text for token in tokenize(text)] for text in df_val["text"].tolist()]
test_texts = [[token.text for token in tokenize(text)] for text in df_test["text"].tolist()]
del df_test
del df_train
del df_val

In [ ]:
vocabulary = Vocabulary()
vocabulary.build(train_texts)
assert vocabulary.word2index[vocabulary.index2word[10]] == 10
print(vocabulary.size)
print(vocabulary.top(20))

17198
['<unk>', 'USER', 'не', 'я', 'и', 'в', 'на', 'rt', 'а', 'что', 'URL', 'с', 'как', 'у', 'все', 'меня', 'то', 'это', 'так', 'мне']


In [ ]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.metrics import Accuracy
from pytorch_lightning.callbacks import EarlyStopping  


class FastTextLSTMModel(LightningModule):
    def __init__(self, ft_embedding_dim, hidden_dim=64):
        super().__init__()

        self.lstm_layer = nn.LSTM(ft_embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.dropout_layer = nn.Dropout(0.2)        
        self.out_layer = nn.Linear(hidden_dim * 2, 1)
        
        self.loss = nn.BCEWithLogitsLoss()
        self.valid_accuracy = Accuracy()
        self.test_accuracy = Accuracy()
    
    def forward(self, embeddings, labels):
        batch_size = embeddings.size(0)
        output, (final_hidden_state, final_cell_state) = self.lstm_layer(embeddings)
        final_hidden_state = final_hidden_state.transpose(0, 1)
        final_hidden_state = final_hidden_state.reshape(batch_size, -1)
        text_hidden = self.dropout_layer(final_hidden_state)
        logits = self.out_layer.forward(text_hidden)
        loss = self.loss(logits, labels)
        return loss, logits
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return [optimizer]
    
    def training_step(self, batch, _):
        inputs, labels = batch
        loss, logits = self(inputs, labels)
        return loss
    
    def validation_step(self, batch, _):
        inputs, labels = batch
        val_loss, logits = self(inputs, labels)
        self.valid_accuracy.update(logits, labels)
        self.log("val_loss", val_loss, prog_bar=True)
        self.log("val_acc", self.valid_accuracy)

    def validation_epoch_end(self, outs):
        self.log("val_acc_epoch", self.valid_accuracy.compute(), prog_bar=True)

    def test_step(self, batch, _):
        inputs, labels = batch
        test_loss, logits = self(inputs, labels)
        self.test_accuracy.update(logits, labels)
        self.log("test_loss", test_loss, prog_bar=True)
        self.log("test_acc", self.test_accuracy)

    def test_epoch_end(self, outs):
        self.log("test_acc_epoch", self.test_accuracy.compute(), prog_bar=True)


## FastText on Tweets

In [ ]:
!wget https://www.dropbox.com/s/t7fthf8axi30hct/ft_ru_tweets_model_v2.bin

--2020-12-16 14:32:09--  https://www.dropbox.com/s/t7fthf8axi30hct/ft_ru_tweets_model_v2.bin
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/t7fthf8axi30hct/ft_ru_tweets_model_v2.bin [following]
--2020-12-16 14:32:09--  https://www.dropbox.com/s/raw/t7fthf8axi30hct/ft_ru_tweets_model_v2.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4f2f929734c7fe0704604a847d.dl.dropboxusercontent.com/cd/0/inline/BFI4wIO_05l21Lar-XLjGQScGErCEq2wSr2VyLseYfVEZUe8QtkMOM3n9OFnlOScVCOUzN9N3g_O4OMZ5FekAKsMXnHDhCOMBbiccaXYkEv59kgPdAtl4ZnfbrT8GaWOmN0/file# [following]
--2020-12-16 14:32:10--  https://uc4f2f929734c7fe0704604a847d.dl.dropboxusercontent.com/cd/0/inline/BFI4wIO_05l21Lar-XLjGQScGErCEq2wSr2VyLseYfVEZUe8QtkMOM3n9OFnlOScVCOUzN9N

In [ ]:
import fasttext
ft_model = fasttext.load_model('ft_ru_tweets_model_v2.bin')

In [ ]:
BATCH_SIZE = 128

ft_train_data = FastTextDataset(train_texts, train_labels, ft_model)
ft_train_sampler = RandomSampler(ft_train_data)
ft_train_loader = DataLoader(ft_train_data, batch_size=BATCH_SIZE, sampler=ft_train_sampler)

ft_val_data = FastTextDataset(val_texts, val_labels, ft_model)
ft_val_loader = DataLoader(ft_val_data, batch_size=BATCH_SIZE)

ft_test_data = FastTextDataset(test_texts, test_labels, ft_model)
ft_test_loader = DataLoader(ft_test_data, batch_size=BATCH_SIZE)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
0
10000
20000
30000
0
10000
20000
30000
40000


In [ ]:
ft_lstm_model = FastTextLSTMModel(128)
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=1,
    verbose=True,
    mode="min" 
)
trainer = Trainer(
    gpus=1,
    checkpoint_callback=False,
    accumulate_grad_batches=1,
    max_epochs=10,
    progress_bar_refresh_rate=10,
    callbacks=[early_stop_callback])
trainer.fit(ft_lstm_model, ft_train_loader, ft_val_loader)
trainer.test(ft_lstm_model, ft_test_loader)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params
-----------------------------------------------------
0 | lstm_layer     | LSTM              | 99.3 K
1 | dropout_layer  | Dropout           | 0     
2 | out_layer      | Linear            | 129   
3 | loss           | BCEWithLogitsLoss | 0     
4 | valid_accuracy | Accuracy          | 0     
5 | test_accuracy  | Accuracy          | 0     
-----------------------------------------------------
99.5 K    Trainable params
0         Non-trainable params
99.5 K    Total params



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.7457, device='cuda:0'),
 'test_acc_epoch': tensor(0.7457, device='cuda:0'),
 'test_loss': tensor(0.4566, device='cuda:0'),
 'val_acc': tensor(0.7523, device='cuda:0'),
 'val_acc_epoch': tensor(0.7523, device='cuda:0'),
 'val_loss': tensor(0.4496, device='cuda:0')}
--------------------------------------------------------------------------------


[{'test_acc': 0.7457449436187744,
  'test_acc_epoch': 0.7457449436187744,
  'test_loss': 0.456634521484375,
  'val_acc': 0.7522615790367126,
  'val_acc_epoch': 0.7522615790367126,
  'val_loss': 0.44955331087112427}]

## FastText on Wiki

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.ru.zip

--2020-12-16 21:04:08--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.ru.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8344906764 (7.8G) [application/zip]
Saving to: ‘wiki.ru.zip’

wiki.ru.zip         100%[===================>]   7.77G  23.5MB/s    in 5m 43s  

2020-12-16 21:09:52 (23.2 MB/s) - ‘wiki.ru.zip’ saved [8344906764/8344906764]



In [ ]:
!unzip wiki.ru.zip -d wiki.ru
!rm wiki.ru.zip
!rm wiki.ru/wiki.ru.vec

Archive:  wiki.ru.zip
  inflating: wiki.ru/wiki.ru.vec     
  inflating: wiki.ru/wiki.ru.bin     


In [ ]:
%%time
import fasttext
ft_model_wiki = fasttext.load_model('./wiki.ru/wiki.ru.bin')

CPU times: user 3.61 s, sys: 6.62 s, total: 10.2 s
Wall time: 3min 17s


In [ ]:
# !rm wiki.ru/wiki.ru.bin

In [ ]:
print(len(train_labels), len(val_texts), len(test_texts))

143260 35816 44770


In [ ]:
ft_wiki_train_data = FastTextDataset(train_texts[:10000], train_labels[:10000], ft_model_wiki)
ft_wiki_val_data = FastTextDataset(val_texts[:1000], val_labels[:1000], ft_model_wiki)
del ft_model_wiki

0
0


In [ ]:
BATCH_SIZE = 128

ft_wiki_train_sampler = RandomSampler(ft_wiki_train_data)
ft_wiki_train_loader = DataLoader(ft_wiki_train_data, batch_size=BATCH_SIZE, sampler=ft_wiki_train_sampler)
del ft_wiki_train_data
ft_wiki_val_loader = DataLoader(ft_wiki_val_data, batch_size=BATCH_SIZE)
del ft_wiki_val_data

In [ ]:
# здесь RAM переполняется и вылетает
ft_wiki_lstm_model = FastTextLSTMModel(128)
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.0,
    patience=1,
    verbose=True,
    mode="min" 
)
trainer = Trainer(
    gpus=1,
    checkpoint_callback=False,
    accumulate_grad_batches=1,
    max_epochs=10,
    progress_bar_refresh_rate=10,
    callbacks=[early_stop_callback])
trainer.fit(ft_wiki_lstm_model, ft_wiki_train_loader, ft_wiki_val_loader)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type              | Params
-----------------------------------------------------
0 | lstm_layer     | LSTM              | 99.3 K
1 | dropout_layer  | Dropout           | 0     
2 | out_layer      | Linear            | 129   
3 | loss           | BCEWithLogitsLoss | 0     
4 | valid_accuracy | Accuracy          | 0     
5 | test_accuracy  | Accuracy          | 0     
-----------------------------------------------------
99.5 K    Trainable params
0         Non-trainable params
99.5 K    Total params


In [ ]:
%%time
import fasttext
ft_model_wiki = fasttext.load_model('./wiki.ru/wiki.ru.bin')

CPU times: user 3.52 s, sys: 6.69 s, total: 10.2 s
Wall time: 3min 18s


In [ ]:
ft_wiki_test_data = FastTextDataset(test_texts[:1000], test_labels[:1000], ft_model_wiki)
ft_wiki_test_loader = DataLoader(ft_wiki_test_data, batch_size=BATCH_SIZE)
trainer.test(ft_wiki_lstm_model, ft_wiki_test_loader)